In [1]:
import pandas as pd
import sqlalchemy
import geopandas as gpd
import seaborn as sns
from shapely import wkt
from itertools import cycle, islice

In [2]:
df = pd.read_csv(r'..\..\..\outputs\network\network_results.csv')

# Congested network components by time of day
df.columns

# Get freeflow from 20to5 period

# Exclude trips taken on non-designated facilities (facility_type == 0)
# These are artificial (weave lanes to connect HOV) or for non-auto uses 
df = df[df['data3'] != 0]    # data3 represents facility_type

# calculate total link VMT and VHT
df['VMT'] = df['@tveh']*df['length']
df['VHT'] = df['@tveh']*df['auto_time']/60

# Define facility type
df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

# Calculate delay
# Select links from overnight time of day
delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

# Merge delay field back onto network link df
df = pd.merge(df, delay_df, on='ij', how='left')

# Calcualte hourly delay
df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

# Calcualte speed as a percent of freeflow
df['flow_index'] = df['auto_time']/df['freeflow_time']

df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})

df['speed'] = df['length']/df['auto_time']*60
df['congestion_index'] = df['speed']/df['data2']
df['congestion_index'] = df['congestion_index'].clip(0,1)
df['congestion_category'] = pd.cut(df['congestion_index'], bins=[0,.25,.5,.7,1], labels=['Severe','Heavy','Moderate','Light'])

In [3]:
def read_from_sde(connection_string, feature_class_name, version,
                  crs={'init': 'epsg:2285'}, is_table = False):
    """
    Returns the specified feature class as a geodataframe from ElmerGeo.
    
    Parameters
    ----------
    connection_string : SQL connection string that is read by geopandas 
                        read_sql function
    
    feature_class_name: the name of the featureclass in PSRC's ElmerGeo 
                        Geodatabase
    
    cs: cordinate system
    """


    engine = sqlalchemy.create_engine(connection_string)
    con=engine.connect()
    #con.execute("sde.set_current_version {0}".format(version))
    if is_table:
        gdf=pd.read_sql('select * from %s' % 
                   (feature_class_name), con=con)
        con.close()

    else:
        df=pd.read_sql('select *, Shape.STAsText() as geometry from %s' % 
                   (feature_class_name), con=con)
        con.close()

        df['geometry'] = df['geometry'].apply(wkt.loads)
        gdf=gpd.GeoDataFrame(df, geometry='geometry')
        gdf.crs = crs
        cols = [col for col in gdf.columns if col not in 
                ['Shape', 'GDB_GEOMATTR_DATA', 'SDE_STATE_ID']]
        gdf = gdf[cols]
    
    return gdf


In [4]:
def format_df(df, col_list, col_name, index_list, index_name):
    
    # Re-order columns
    df = df[col_list]
    # Re-order index
    df = df.reindex(index_list)
    # Set visible column and index names
    df.columns.name = col_name
    df.index.name = index_name
    
    return df


In [5]:
# set chart style
sns.set_style("dark")
sns.set_context("notebook", font_scale=1.5, 
                rc={"lines.linewidth": 1.5})
colors = ['#91268F','#F05A28','#8CC63E','#00A7A0','#4C4C4C', '#BFE9E7']
sns.set(font="Pragmatica Condensed Light")

In [6]:
# Load regional geographies layer to intersect with network shapefile
connection_string = 'mssql+pyodbc://AWS-PROD-SQL\Sockeye/ElmerGeo?driver=SQL Server?Trusted_Connection=yes'
crs = {'init' : 'EPSG:2285'}
version = "'DBO.Default'"
gdf_shp = read_from_sde(connection_string, 'urban_centers', version, crs=crs, is_table=False)

# Intersect geography this with the network shapefile
gdf_network = gpd.read_file(r'..\..\..\inputs\scenario\networks\shapefiles\AM\AM_edges.shp')
gdf_intersect = gpd.overlay(gdf_network, gdf_shp, how="union", keep_geom_type=False)

# Will need to relaculate the lengths since some were split across the regional geographies
gdf_intersect['new_length'] = gdf_intersect.geometry.length/5280.0

### IMPORTANT
# filter out the polygon results and only keep lines
gdf_intersect = gdf_intersect[gdf_intersect.geometry.type.isin(['MultiLineString','LineString'])]

C:\Users\Modeller\AppData\Local\anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [7]:
df = df[['@tveh','congestion_category','i_node','j_node','tod','total_delay']].merge(gdf_intersect, 
                                                             left_on=['i_node','j_node'],
                                                            right_on=['i','j'],
                                                                  how='left')
df['VMT'] = df['new_length']*df['@tveh']

# Assume speed is constant or assigned link
# When links are split, we take proportionate delay by length

# calculate total delay
df['delay'] = (df['new_length']/df['length'])*df['total_delay']
# Note that we won't get all of the data because some links are outside the region boundaries
# Optionally, scale these numbers by the total delay field total_delay for consistency
df['scaled_delay'] = df['delay']*(df['total_delay'].sum()/df['delay'].sum())

# Congestion by Regional Geography
### Total VMT & Percent of VMT by Congestion Level

- **Severe: < 25% Speed Limit** (e.g., less than 15 mph on a road with speed limit of 60 mph)
- **Heavy: 25-50% Speed Limit** (e.g., 15-30 mph on a road with speed limit of 60 mph)
- **Moderate: 50-70% Speed Limit** (e.g, 30-42 mph on a road with speed limit of 60 mph)

#### All Day

In [8]:
df.rename(columns={'name': 'Regional Growth Center'}, inplace=True)
df['Regional Growth Center'] = df['Regional Growth Center'].fillna('Not in RGC')

# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.pivot_table(index='congestion_category',columns='Regional Growth Center',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,264,"3,617","9,895","37,295"
Bellevue,"1,161","7,216","18,997","62,215"
Bothell Canyon Park,"1,715","12,436","23,397","126,275"
Bremerton,71,683,"2,494","14,909"
Burien,250,"2,003","8,922","65,445"
Everett,0,"8,193","19,045","66,703"
Federal Way,62,"1,034","3,762","20,203"
Greater Downtown Kirkland,690,"11,065","20,211","114,677"
Issaquah,"1,999","3,762","11,615","183,009"


In [9]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,0.5%,7.1%,19.4%,73.0%
Bellevue,1.3%,8.1%,21.2%,69.4%
Bothell Canyon Park,1.0%,7.6%,14.3%,77.1%
Bremerton,0.4%,3.8%,13.7%,82.1%
Burien,0.3%,2.6%,11.6%,85.4%
Everett,0.0%,8.7%,20.3%,71.0%
Federal Way,0.2%,4.1%,15.0%,80.6%
Greater Downtown Kirkland,0.5%,7.5%,13.8%,78.2%
Issaquah,1.0%,1.9%,5.8%,91.3%


#### AM Peak (7 to 8 AM)

In [10]:
pd.options.display.float_format = '{:0,.0f}'.format
df_am = df[df['tod'] == '7to8']
_df = df_am.pivot_table(index='congestion_category',columns='Regional Growth Center',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,25,391,"1,384","1,775"
Bellevue,63,395,"1,174","4,185"
Bothell Canyon Park,109,"1,466","3,043","6,160"
Bremerton,2,57,159,"1,124"
Burien,14,225,379,"4,176"
Everett,0,684,"1,497","4,312"
Federal Way,4,5,282,"1,086"
Greater Downtown Kirkland,43,"2,029","1,429","6,538"
Issaquah,332,250,"2,359","12,211"


In [11]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,0.7%,10.9%,38.7%,49.7%
Bellevue,1.1%,6.8%,20.2%,71.9%
Bothell Canyon Park,1.0%,13.6%,28.2%,57.2%
Bremerton,0.1%,4.3%,11.9%,83.7%
Burien,0.3%,4.7%,7.9%,87.1%
Everett,0.0%,10.5%,23.1%,66.4%
Federal Way,0.3%,0.4%,20.5%,78.9%
Greater Downtown Kirkland,0.4%,20.2%,14.2%,65.1%
Issaquah,2.2%,1.7%,15.6%,80.6%


#### PM Peak (5 to 6 PM)

In [12]:
pd.options.display.float_format = '{:0,.0f}'.format
df_pm = df[df['tod'] == '17to18']
_df = df_pm.pivot_table(index='congestion_category',columns='Regional Growth Center',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,23,370,743,"3,073"
Bellevue,100,894,"1,579","4,921"
Bothell Canyon Park,260,"1,227","2,687","8,462"
Bremerton,10,106,176,"1,247"
Burien,20,263,856,"5,610"
Everett,0,849,"1,444","5,696"
Federal Way,5,111,271,"1,426"
Greater Downtown Kirkland,65,"2,304","2,412","7,082"
Issaquah,193,377,"2,239","14,389"


In [13]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
Regional Growth Center,,,,
Auburn,0.5%,8.8%,17.7%,73.0%
Bellevue,1.3%,11.9%,21.1%,65.7%
Bothell Canyon Park,2.1%,9.7%,21.3%,67.0%
Bremerton,0.6%,6.9%,11.4%,81.0%
Burien,0.3%,3.9%,12.7%,83.1%
Everett,0.0%,10.6%,18.1%,71.3%
Federal Way,0.3%,6.1%,14.9%,78.7%
Greater Downtown Kirkland,0.5%,19.4%,20.3%,59.7%
Issaquah,1.1%,2.2%,13.0%,83.7%


# Network Delay
Totals scaled to match network totals. 

In [14]:
# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.groupby('Regional Growth Center').sum()[['scaled_delay']]
_df.loc['Total',:] = _df.sum() 
_df

,scaled_delay
Regional Growth Center,
Auburn,164
Bellevue,267
Bothell Canyon Park,884
Bremerton,17
Burien,87
Everett,176
Federal Way,23
Greater Downtown Kirkland,561
Issaquah,631


Totals unscaled. Some links cross outside the region and delay on these links are excluded at boundaries

In [15]:
# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.groupby('Regional Growth Center').sum()[['delay']]
_df.loc['Total',:] = _df.sum() 
_df

,delay
Regional Growth Center,
Auburn,154
Bellevue,252
Bothell Canyon Park,831
Bremerton,16
Burien,82
Everett,165
Federal Way,21
Greater Downtown Kirkland,528
Issaquah,594
